In [1]:
%matplotlib inline
import numpy as np
import pandas as pd

from scipy.io import loadmat 

import matplotlib.pyplot as plt
import seaborn as sns
sns.reset_orig()

In [2]:
import networkx as nx

from scipy.stats import bernoulli
from scipy.special import loggamma

from sklearn.preprocessing import OneHotEncoder

import tqdm

from rpy2.robjects import numpy2ri
from rpy2.robjects.packages import importr

In [3]:
import os
import glob
import pickle

In [4]:
blockmodels = importr("blockmodels")

In [5]:
base = importr("base")

In [6]:
dollar = base.__dict__["$"]

In [7]:
pd.options.display.max_rows = 1000

In [8]:
outdir = './output'
if not os.path.exists(outdir):
    os.makedirs(outdir)

In [9]:
from functools import lru_cache

@lru_cache(maxsize = 10000)
def normterm_discrete(n, k):
    if n == 1:
        return np.log(k)
    if k == 1:
        return 1.0
    elif k == 2:
        return np.sum(sorted([ np.exp(loggamma(n+1) - loggamma(t+1) - loggamma(n-t+1) + 
                               t*(np.log(t) - np.log(n)) + (n-t)*(np.log(n-t) - np.log(n))
                        )
                        for t in range(1, n)]))
    else:
        return normterm_discrete(n, k-1) + n/(k-2) * normterm_discrete(n, k-2)

In [10]:
def calc_dnml(X, Z1, Z2, K=3, L=3):    
    N = X.shape[0]
    
    codelen_x_z = 0.0
    codelen_z = 0.0

    for k in range(K):
        for l in range(L):
            n_pos = np.sum(X[Z1 == k, :][:, Z2 == l] == 1)
            n_neg = np.sum(X[Z1 == k, :][:, Z2 == l] == 0)
            n_all = n_pos + n_neg

            if n_all >=2:
                codelen_x_z += n_all * np.log(n_all)
                codelen_x_z += np.log(normterm_discrete(n_all, 2))
            
            if n_pos >=2:
                codelen_x_z -= n_pos * np.log(n_pos)
            if n_neg >=2:
                codelen_x_z -= n_neg * np.log(n_neg)
            
        n_k = np.sum(Z1 == k)
        if n_k >= 1:
            codelen_z += n_k * (np.log(N) - np.log(n_k))

    codelen_z += np.log(normterm_discrete(N, K))
    
    codelen = codelen_x_z + codelen_z

    return codelen, codelen_x_z, codelen_z

In [11]:
def calc_lsc(X, Z1, Z2, K=3, L=3):    
    codelen = 0.0
    N = X.shape[0]

    for k in range(K):
        for l in range(L):
            n_pos = np.sum(X[Z1 == k, :][:, Z2 == l] == 1)
            n_neg = np.sum(X[Z1 == k, :][:, Z2 == l] == 0)
            n_all = n_pos + n_neg

            if n_all >=2:
                codelen += n_all * np.log(n_all)
            
            if n_pos >=2:
                codelen -= n_pos * np.log(n_pos)
            if n_neg >=2:
                codelen -= n_neg * np.log(n_neg)

        n_k = np.sum(Z1 == k)
        if n_k >= 1:
            codelen += n_k * (np.log(N) - np.log(n_k))

        codelen += (k + (k+1)*(k+2))/2 * np.log(N/(2.0*np.pi)) -(k+1)/2 * np.log(2.0) + \
           (k+1) * loggamma((k+3)/2) - loggamma((k+1)*(k+3)/2) + (k+1)*(k+2)/2 * np.log(np.pi)

    return codelen

In [12]:
def estimate_sbm(X, cluster_range_max=6):
    numpy2ri.activate()
    sbm = blockmodels.BM_bernoulli(membership_type="SBM", adj=np.array(X),
                                       verbosity=0,
                                       exploration_factor=1.5,
                                       explore_min=cluster_range_max,
                                       explore_max=cluster_range_max)

    estimate = dollar(sbm, "estimate")
    estimate()

    theta_list = []
    pi_list = []
    z_posterior_list = []
    for k in range(cluster_range_max):
        n_clusters = k + 1
        theta = np.array(dollar(dollar(sbm, "model_parameters")[k], "pi"))
        z_posterior = np.array(dollar(dollar(sbm, "memberships")[k], "Z"))
        pi = np.sum(z_posterior, axis=0) + 10 * EPS
        pi /= np.sum(pi)

        theta_list.append(theta)
        z_posterior_list.append(z_posterior)
        pi_list.append(pi)

    return pi_list, theta_list, z_posterior_list

In [13]:
cluster_range_max = 7

In [14]:
files_pairs = sorted(glob.glob('./data/enron/*.pairs'))

In [15]:
with open('addresses.txt', 'r') as f:
    addresses = np.array(f.read().splitlines())

In [16]:
n_term = len(files_pairs)
n_employee = len(addresses)
X = np.zeros((n_employee, n_employee, n_term), dtype=np.int)

for k, file in enumerate(files_pairs):
    print(file)
    try:
        sender_receiver = np.loadtxt(file, delimiter='\t', dtype=np.int)
        if len(sender_receiver.shape) == 1:
            sender_receiver = sender_receiver.reshape(1, -1)
        id_sender = sender_receiver[:, 0] - 1
        id_receiver = sender_receiver[:, 1] - 1
        X[id_sender, id_receiver, k] += 1
    except:
        continue

./data/enron/19990510.pairs
./data/enron/19990517.pairs
./data/enron/19990524.pairs
./data/enron/19990531.pairs
./data/enron/19990607.pairs
./data/enron/19990614.pairs
./data/enron/19990621.pairs
./data/enron/19990628.pairs
./data/enron/19990705.pairs
./data/enron/19990712.pairs
./data/enron/19990719.pairs
./data/enron/19990726.pairs
./data/enron/19990802.pairs
./data/enron/19990809.pairs
./data/enron/19990816.pairs
./data/enron/19990823.pairs
./data/enron/19990830.pairs
./data/enron/19990906.pairs
./data/enron/19990913.pairs
./data/enron/19990920.pairs
./data/enron/19990927.pairs
./data/enron/19991004.pairs
./data/enron/19991011.pairs
./data/enron/19991018.pairs
./data/enron/19991025.pairs
./data/enron/19991101.pairs
./data/enron/19991108.pairs
./data/enron/19991115.pairs
./data/enron/19991122.pairs
./data/enron/19991129.pairs
./data/enron/19991206.pairs
./data/enron/19991213.pairs
./data/enron/19991220.pairs
./data/enron/19991227.pairs
./data/enron/20000103.pairs
./data/enron/2000011

/Users/shi-fukushima/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: loadtxt: Empty input file: "./data/enron/20020513.pairs"
  
/Users/shi-fukushima/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: loadtxt: Empty input file: "./data/enron/20020520.pairs"
  
/Users/shi-fukushima/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: loadtxt: Empty input file: "./data/enron/20020603.pairs"
  


In [17]:
X.shape

(151, 151, 162)

In [18]:
with open('n_emails.pkl', 'wb') as f:
    pickle.dump(X, f)

In [19]:
X = (X > 0).astype(np.int32)

In [20]:
np.sum(X)

7444

In [21]:
EPS = np.finfo(np.float).eps

#T = X_all.shape[1]
T = 157

h = 2

pi_all_total = []
theta_all_total = []
z_all_total = []

pi_all_former = []
theta_all_former = []
z_all_former = []

pi_all_latter = []
theta_all_latter = []
z_all_latter = []
        
for t in tqdm.tqdm(range(h, T-h)):
    seed = t

    # total
    X_total = np.any(X[:, :, (t-h):(t+h)] == 1, axis=2).astype(np.int)
    print(X_total.shape)
    pi_list, theta_list, z_posterior_list = estimate_sbm(X_total, cluster_range_max)            

    pi_all_total.append(pi_list)
    theta_all_total.append(theta_list)
    z_all_total.append(z_posterior_list)

    # former
    X_former = np.any(X[:, :, (t-h):t] == 1, axis=2).astype(np.int)
    pi_list, theta_list, z_posterior_list = estimate_sbm(X_former, cluster_range_max)

    pi_all_former.append(pi_list)
    theta_all_former.append(theta_list)
    z_all_former.append(z_posterior_list)

    # latter
    X_latter = np.any(X[:, :, t:(t+h)] == 1, axis=2).astype(np.int)
    pi_list, theta_list, z_posterior_list = estimate_sbm(X_latter, cluster_range_max)

    pi_all_latter.append(pi_list)
    theta_all_latter.append(theta_list)
    z_all_latter.append(z_posterior_list)

with open(os.path.join(outdir, 'pi_total_enron.pkl'), 'wb') as f:
    pickle.dump(pi_all_total, f)
with open(os.path.join(outdir, 'theta_total_enron.pkl'), 'wb') as f:
    pickle.dump(theta_all_total, f)
with open(os.path.join(outdir, 'z_total_enron.pkl'), 'wb') as f:
    pickle.dump(z_all_total, f)

with open(os.path.join(outdir, 'pi_former_enron.pkl'), 'wb') as f:
    pickle.dump(pi_all_former, f)
with open(os.path.join(outdir, 'theta_former_enron.pkl'), 'wb') as f:
    pickle.dump(theta_all_former, f)
with open(os.path.join(outdir, 'z_former_enron.pkl'), 'wb') as f:
    pickle.dump(z_all_former, f)

with open(os.path.join(outdir, 'pi_latter_enron.pkl'), 'wb') as f:
    pickle.dump(pi_all_latter, f)
with open(os.path.join(outdir, 'theta_latter_enron.pkl'), 'wb') as f:
    pickle.dump(theta_all_latter, f)
with open(os.path.join(outdir, 'z_latter_enron.pkl'), 'wb') as f:
    pickle.dump(z_all_latter, f)

  0%|          | 0/153 [00:00<?, ?it/s]

(151, 151)





  1%|          | 1/153 [00:07<19:52,  7.85s/it]

(151, 151)





  1%|▏         | 2/153 [00:14<18:37,  7.40s/it]

...

 99%|█████████▊| 151/153 [22:38<00:09,  4.56s/it]

(151, 151)





 99%|█████████▉| 152/153 [22:43<00:04,  4.60s/it]

(151, 151)





100%|██████████| 153/153 [22:48<00:00,  4.62s/it]
